In [1]:
# to transform XES to PKL
import os
import glob
import pandas as pd
# from dateutil.parser import parse
from tqdm import tqdm
import pm4py
from collections import Counter


folder_path = 'data'  # Replace with the path to your folder

# Use glob to get a list of all .xes files in the folder
xes_files = glob.glob(os.path.join(folder_path, '*.xes'))

# Print the list of .xes files
for xes_file in xes_files:
    file_name = os.path.basename(xes_file)
    print(file_name)
    

    
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

do_display=True

def build_entity_sequence(df_input,entity_colname,obs_colname,time_colname,method_time):
    """Crée un jeu de données de séquence, a la maille entité-> passage de la maille "obs/entité" a la maille "entité"
    Parameters
    ----------
    df : dataframe
        La dataframe utilisée
        
    Returns
    ----------
    df_seq : dataframe
        jeu de données de séquences à la maille entité
    """ 
    
    def create_obs_sequence_for_entity(df_indiv): # crée la séquence d'événements pour l'entité
        seq_entite = []
        for j in range(len(df_indiv)):
            seq_entite.append(df_indiv.loc[j, obs_colname])
        return seq_entite

    def create_interval_sequence_for_entity(df_indiv): # utilise les intervalles de temps dans la séquence timestamps
        interv_seq_entite = []
        for j in range(len(df_indiv)):
            if j==len(df_indiv)-1:
                interv_seq_entite.append(0.0)
            else:
                interv_seq_entite.append(df_indiv.loc[j+1, time_colname]-df_indiv.loc[j, time_colname])
        return interv_seq_entite
    
    def create_timestamp_sequence_for_entity(df_indiv): # utilise les timestamps dans la séquence timestamps
        interv_seq_entite = []
        for index, row in df_indiv.iterrows():
            interv_seq_entite.append(row["date"])
        return interv_seq_entite
    def create_empty_time_sequence_for_entity(df_indiv): # remplace la séquence de timestamps par des 0
        interv_seq_entite = []
        for index, row in df_indiv.iterrows():
            interv_seq_entite.append(0)
        return interv_seq_entite
    df=df_input.copy()
    min_date = df['date'].min()
    df['date']=(df.date-min_date).dt.total_seconds()/3600 # timestamp to number of seconds since first obs
    
    dicCol = {x:y for x,y in df.groupby(entity_colname)}
    
    # filter short sequences
    min_length = 1
    dicCol = {key: df for key, df in dicCol.items() if len(df) >= min_length} 
    # print(len(dicCol))
    
    list_seq = []
    columns = ['Entite', 'Obs_seq', 'Intervals_seq']
    
    for entite, df_entite in tqdm(dicCol.items()): 
        
        df_entite = df_entite.sort_values(by=['date'], ignore_index=True)
        
        seq_entite = create_obs_sequence_for_entity(df_entite)
        
        if method_time=="timestamp":
            seq_entite_time = create_timestamp_sequence_for_entity(df_entite)
            
        elif method_time=="interval":
            seq_entite_time = create_interval_sequence_for_entity(df_entite)
            
        elif method_time=="no_timestamp":
            seq_entite_time = create_empty_time_sequence_for_entity(df_entite)
            
        else:
            return None
        
        list_seq.append([entite, seq_entite, seq_entite_time])

    # Créez le DataFrame une fois que toutes les données sont prêtes
    df_seq_entite = pd.DataFrame(list_seq, columns=columns)
    return df_seq_entite

def get_variable_frequency_stats(data,col_name):
    min_date=data['date'].min()
    datax = data[col_name].value_counts().sort_index()
    datay = pd.DataFrame({
      'state': datax.index,
      'Frequency': datax.values,
      'Percent': ((datax.values/datax.values.sum())*100).round(2),
      'Cumulative Frequency': datax.values.cumsum(),
      'Cumulative Percent': ((datax.values.cumsum()/datax.values.sum())*100)\
    .round(2)
    })
    display(datay.sort_values(by=['Frequency']))
    return None



do_display=True
list_dataset = [
    
    "env_permit",
    "Helpdesk",
    "nasa",
    "SEPSIS",
    "BPI_Challenge_2012",
    "BPI_Challenge_2012_A",
    "BPI_Challenge_2012_Complete",
    "BPI_Challenge_2012_O",
    "BPI_Challenge_2012_W",
    "BPI_Challenge_2012_W_Complete",
    "BPI_Challenge_2013_closed_problems",
    "bpi_challenge_2013_incidents",
    "BPI Challenge 2017",
    "BPI_Challenge_2019",

    # "BPIC15_1",
#     "BPIC15_2",
#     "BPIC15_3",
#     "BPIC15_4",
#     "BPIC15_5",
#     "Hospital_log"
]
list_dataset = ["DATA_MCF_1an_prest"]

for file_name in list_dataset: 
    print("======================================")
    print("-------- Dataset: "+file_name)
    print("======================================")
    data = pm4py.read_xes('data/%s.xes'%(file_name))
    data = pm4py.convert_to_dataframe(data)[["case:concept:name","time:timestamp","concept:name"]]
    data.rename(columns={'time:timestamp':'date'}, inplace=True)
    data.rename(columns={'case:concept:name':'entity'}, inplace=True)
    data.rename(columns={'concept:name':'type'}, inplace=True)
    
    if do_display: display(data.head(10))
    data=data[data['entity'].notnull()]
    data["date"] = pd.to_datetime(data["date"])
    data['entity'] = pd.factorize(data['entity'])[0]
    data['type'] = pd.factorize(data['type'])[0]
    
    if do_display: print('df_entity_seq_interval')
    df_entity_seq_interval     = build_entity_sequence(data,'entity',"type","date",method_time="interval") # appel de la fonction ci-dessus avec le param_tre "intervalles"
    if do_display: display(df_entity_seq_interval.head(10))
    df_entity_seq_interval.to_pickle("data/%s.pkl"%(file_name))
    
 

C:\Users\tkast\anaconda3\lib\site-packages\pm4py\util\dt_parsing\parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
C:\Users\tkast\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BPI Challenge 2017.xes
BPIC15_1.xes
BPIC15_2.xes
BPIC15_3.xes
BPIC15_4.xes
BPIC15_5.xes
BPI_Challenge_2012.xes
BPI_Challenge_2012_A.xes
BPI_Challenge_2012_Complete.xes
BPI_Challenge_2012_O.xes
BPI_Challenge_2012_W.xes
BPI_Challenge_2012_W_Complete.xes
BPI_Challenge_2013_closed_problems.xes
bpi_challenge_2013_incidents.xes
BPI_Challenge_2019.xes
env_permit.xes
Helpdesk.xes
Hospital_log.xes
nasa.xes
SEPSIS.xes
-------- Dataset: env_permit


parsing log, completed traces :: 100%|██████████| 1434/1434 [00:00<00:00, 5855.57it/s]


,entity,date,type
0,case-891,2010-10-02 07:20:39.266000+00:00,Confirmation of receipt
1,case-891,2010-10-02 07:21:26.588000+00:00,T02 Check confirmation of receipt
2,case-891,2010-10-02 07:31:12.836000+00:00,T03 Adjust confirmation of receipt
3,case-891,2010-10-02 07:31:40.160000+00:00,T02 Check confirmation of receipt
4,case-891,2010-10-02 07:32:01.401000+00:00,T03 Adjust confirmation of receipt
5,case-891,2010-11-09 13:13:52.563000+00:00,T02 Check confirmation of receipt
6,case-891,2010-11-09 13:14:10.016000+00:00,T04 Determine confirmation of receipt
7,case-891,2010-11-09 13:14:22.628000+00:00,T05 Print and send confirmation of receipt
8,case-891,2010-11-09 13:14:37.300000+00:00,T06 Determine necessity of stop advice
9,case-891,2010-11-09 13:17:36.097000+00:00,T07-1 Draft intern advice aspect 1


df_entity_seq_interval


...

parsing log, completed traces :: 100%|██████████| 251734/251734 [00:57<00:00, 4371.27it/s]


,entity,date,type
0,2000000000_00001,2018-01-02 12:53:00+00:00,SRM: Created
1,2000000000_00001,2018-01-02 13:53:00+00:00,SRM: Complete
2,2000000000_00001,2018-01-02 13:53:00+00:00,SRM: Awaiting Approval
3,2000000000_00001,2018-01-02 13:53:00+00:00,SRM: Document Completed
4,2000000000_00001,2018-01-02 13:53:00+00:00,SRM: In Transfer to Execution Syst.
5,2000000000_00001,2018-01-02 13:53:00+00:00,SRM: Ordered
6,2000000000_00001,2018-01-02 13:53:00+00:00,SRM: Change was Transmitted
7,2000000000_00001,2018-01-02 13:53:00+00:00,Create Purchase Order Item
8,2000000000_00001,2018-01-02 22:59:00+00:00,Vendor creates invoice
9,2000000000_00001,2018-03-06 06:44:00+00:00,Record Goods Receipt


df_entity_seq_interval


100%|██████████| 251734/251734 [00:56<00:00, 4458.90it/s]


,Entite,Obs_seq,Intervals_seq
0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.09999999..."
1,1,"[0, 1, 2, 3, 5, 4, 6, 7, 8, 9, 12, 10, 11, 4, 13]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 181.166666..."
2,2,"[0, 1, 2, 3, 5, 4, 6, 7, 12, 9, 8, 14, 9, 12, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 792.599999..."
3,3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.6333333..."
4,4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.6333333..."
5,5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.6333333..."
6,6,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.6333333..."
7,7,"[0, 1, 2, 3, 4, 5, 7, 6, 8, 15, 9, 10, 11, 16,...","[1.0, 0.0, 0.0, 0.016666666604578495, 0.0, 0.0..."
8,8,"[0, 1, 2, 3, 4, 5, 7, 6]","[1.0, 0.0, 0.0, 0.016666666604578495, 0.0, 0.0..."
9,9,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 10, 11, 4, 17]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.4333333..."
